In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

base_dir= "/kaggle/input/flickr-image-dataset/flickr30k_images/"
dataset = "/kaggle/input/flickr-image-dataset/flickr30k_images/results.csv"
IMG_PATH = "/kaggle/input/flickr-image-dataset/flickr30k_images/flickr30k_images/"

In [3]:
%pip install --upgrade tensorflow_hub
import tensorflow_hub as hub

Note: you may need to restart the kernel to use updated packages.



In [5]:
import pandas as pd
import tensorflow as tf
import tensorflow_text

ModuleNotFoundError: No module named 'tensorflow_text'

In [6]:
print(dataset)

/kaggle/input/flickr-image-dataset/flickr30k_images/results.csv


In [7]:
df=pd.read_csv(dataset, delimiter='|')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/flickr-image-dataset/flickr30k_images/results.csv'

In [8]:
from tqdm import tqdm, tqdm_notebook
tqdm.pandas()

In [9]:
df.columns
df.fillna

NameError: name 'df' is not defined

In [10]:
img_name_train=[]
cap_train=[]
for index, row in df.iterrows():
    img_name_train.append(IMG_PATH+row['image_name'])
    cap_train.append(str(row[' comment']))

NameError: name 'df' is not defined

In [11]:
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [12]:
print(len(img_name_train),len(cap_train))

0 0


In [13]:
print(img_name_train[0],cap_train[0])

IndexError: list index out of range

In [14]:
def normalization(embeds):
    norms = np.linalg.norm(embeds, 2, axis=1, keepdims=True)
    return embeds/norms
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
encoder = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-base-br/1")

RuntimeError: Op type not registered 'RegexSplitWithOffsets' in binary running on LAKSHYA. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib (e.g. `tf.contrib.resampler`), accessing should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.

In [ ]:
BATCH_SIZE=5
target_size=(128,128)
embedding_dim=128

In [ ]:
def decode_image(filename, label=None, image_size=(target_size[0],target_size[1])):
    means = [0.485, 0.456, 0.406]
    stds = [0.229, 0.224, 0.225]
    
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    
    image = (tf.cast(image, tf.float32) / 255.0)
    image = (image - means) / stds # for qubvel EfficientNet
    
    image = tf.image.resize(image, image_size)
    
    if label is None:
        return image
    else:
        return image, label

In [ ]:
def data_augment(image, label=None):
    image = tf.image.random_flip_left_right(image)
    
    if label is None:
        return image
    else:
        return image, label

In [ ]:
def get_training_dataset():
    train_dataset = (
        tf.data.Dataset
        .from_tensor_slices((img_name_train, cap_train))
        .map(decode_image, num_parallel_calls=10)
        .cache()
        .map(data_augment, num_parallel_calls=10)
        .repeat() # Maybe not repeat in custom training (so when and how??) <-- the current version is bug because it repeat indefinitely
        .shuffle(BATCH_SIZE*8, reshuffle_each_iteration=True)
        .batch(BATCH_SIZE, drop_remainder=False)
        .prefetch(10)
    )
    return strategy.experimental_distribute_dataset(train_dataset)

In [ ]:
imageEncoderLayer=hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_m/feature_vector/2",
                   trainable=False)


In [ ]:
class VisionEncoder(tf.keras.layers.Layer):
    def __init__(self):
        super(VisionEncoder,self).__init__()
        self.encoder=imageEncoderLayer
        self.ds=tf.keras.layers.Dense(embedding_dim,activation="relu")
    def call(self, x):
        x=self.encoder(x)
        x=self.ds(x)
        return x

In [ ]:
class TextEncoder(tf.keras.layers.Layer):
    def __init__(self,preprocessor,encoder):
        super(TextEncoder,self).__init__()
        self.preprocessor=preprocessor
        self.encoder=encoder
        self.ds=tf.keras.layers.Dense(embedding_dim,activation="relu")
    def call(self, x):
        x=self.preprocessor(x)
        x=self.encoder(x)['default']
        x=normalization(x)
        x=self.ds(x)
        return x

In [ ]:
ds=get_training_dataset()

In [ ]:
num_heads=10
LR=0.001

In [ ]:
class CrossAttention(tf.keras.layers.Layer):
    def __init__(self):
        super(CrossAttention, self).__init__()
        self.query=tf.keras.layers.Dense(embedding_dim,activation="relu")
        self.key=tf.keras.layers.Dense(embedding_dim,activation="relu")
        self.value=tf.keras.layers.Dense(embedding_dim,activation="relu")
        self.outputs=tf.keras.layers.Dense(embedding_dim)

    def call(self, inputs):
        vision_inputs,text_inputs=inputs
        query_inputs=self.query(vision_inputs)
        key_inputs=self.key(text_inputs)
        value_inputs=self.value(text_inputs)
        attention_scores = tf.matmul(query_inputs,key_inputs, transpose_b=True)
        attention_scores = tf.nn.tanh(attention_scores)
        attention_weights = tf.nn.softmax(attention_scores, axis=-1)
        attended = tf.matmul(attention_weights,value_inputs)
        output = tf.concat([text_inputs,attended],axis=-1)
        output=self.outputs(output)
        return output

In [ ]:
class CrossAttentionEncoder(tf.keras.Model):
    def __init__(self,visionEncoder,textEncoder):
        super(CrossAttentionEncoder,self).__init__()
        self.visionEncoder=visionEncoder
        self.textEncoder=textEncoder
        self.cross_attention=CrossAttention()
        self.cross_attention2=CrossAttention()
        self.similarity=tf.keras.losses.CosineSimilarity(axis=1)
    def call(self,inputs):
        vision_input,text_input=inputs
#         print(vision_input,text_input)
        vision_input=self.visionEncoder(vision_input)
        text_input=self.textEncoder(text_input)
        output1=self.cross_attention((vision_input,text_input))
        output2=self.cross_attention2((text_input,vision_input))
        return output1,output2

In [ ]:
import gc
gc.collect()

3

In [ ]:
# with strategy.scope():
textEncoder = TextEncoder(preprocessor,encoder)
ImageEncoder=VisionEncoder()
transformer=CrossAttentionEncoder(ImageEncoder,textEncoder)
optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
loss_object = tf.keras.losses.CosineSimilarity()
    

In [ ]:

def train_step(img_tensor, text):
    loss=0
    hidden=transformer.reset_states()
    with tf.GradientTape() as tape:
        emb_a, emb_b=transformer((img_tensor,text))
        loss+=1-loss_object(emb_a,emb_b)
    total_loss=loss
    trainable_variables=transformer.trainable_variables
    gradient=tape.gradient(loss,trainable_variables)
    optimizer.apply_gradients(zip(gradient,trainable_variables))
    return loss, total_loss
def distributed_train_step(inputs):
    (images, labels) = inputs
    loss = strategy.run(train_step, args=(images, labels))
    return loss

In [ ]:
# with strategy.scope():
counter=0
total_loss=0
for batch in ds:
    if counter==1000:
        break
    counter+=1
    _, t_loss=train_step(batch[0],batch[1])
    total_loss+=t_loss
    print ('Epoch {} Batch {} Loss {:.4f}'.format(1, counter, t_loss.numpy() ))
    
    

Epoch 1 Batch 1 Loss 1.1438
Epoch 1 Batch 2 Loss 0.6787
Epoch 1 Batch 3 Loss 0.4659
Epoch 1 Batch 4 Loss 0.3433
Epoch 1 Batch 5 Loss 0.2673
Epoch 1 Batch 6 Loss 0.2304
Epoch 1 Batch 7 Loss 0.1534
Epoch 1 Batch 8 Loss 0.1292
Epoch 1 Batch 9 Loss 0.0952
Epoch 1 Batch 10 Loss 0.1087
Epoch 1 Batch 11 Loss 0.0858
Epoch 1 Batch 12 Loss 0.0674
Epoch 1 Batch 13 Loss 0.0519
Epoch 1 Batch 14 Loss 0.0456
Epoch 1 Batch 15 Loss 0.0491
Epoch 1 Batch 16 Loss 0.0435
Epoch 1 Batch 17 Loss 0.0440
Epoch 1 Batch 18 Loss 0.0329
Epoch 1 Batch 19 Loss 0.0322
Epoch 1 Batch 20 Loss 0.0254
Epoch 1 Batch 21 Loss 0.0262
Epoch 1 Batch 22 Loss 0.0271
Epoch 1 Batch 23 Loss 0.0245
Epoch 1 Batch 24 Loss 0.0646
Epoch 1 Batch 25 Loss 0.1196
Epoch 1 Batch 26 Loss 0.0601
Epoch 1 Batch 27 Loss 0.0471
Epoch 1 Batch 28 Loss 0.0316
Epoch 1 Batch 29 Loss 0.0187
Epoch 1 Batch 30 Loss 0.0406
Epoch 1 Batch 31 Loss 0.0302
Epoch 1 Batch 32 Loss 0.0205
Epoch 1 Batch 33 Loss 0.0226
Epoch 1 Batch 34 Loss 0.0199
Epoch 1 Batch 35 Loss 0

In [15]:
file_names=[decode_image('/kaggle/input/flickr-image-dataset/flickr30k_images/flickr30k_images/1000092795.jpg')]
text=['Two young guys with shaggy hair look at their hands while hanging out in the yard .']
a,b=transformer((file_names,text))
print(tf.keras.losses.CosineSimilarity()(a,b))
a,b=transformer((file_names,['.3']))
print(tf.keras.losses.CosineSimilarity()(a,b))

NameError: name 'decode_image' is not defined